# **0.  Khai báo đường dẫn**

In [19]:
# Dataset Path
dataset_path = "E:/An toàn thông tin/Code/Dataset/"

# Dataset preprocessed including attack-type labels
trainsets_path = dataset_path + "NSL-KDD Processed/Separate by Category/Encoded&Normalized/Trainset Separate to 2 Parts/"
testsets_path = dataset_path + "NSL-KDD Processed/Separate by Category/Encoded&Normalized/Testset/"

# Dataset Final Path
final_dataset_path = dataset_path + "NSL-KDD Processed/Final - For Using/"

#**1. Chuẩn bị**

##**1.1 Import thư viện**

In [20]:
# Import Lib
import pandas as pd
import numpy as np
from sklearn.utils import shuffle

##**1.2. Load dataset**
Load từng phần của mỗi loại dữ liệu, từ dữ liệu đã chia mỗi category thành 2 phần bằng nhau.

In [21]:
# define categories
train_categories = ['Normal', 'DOS', 'U2R', 'R2L']
test_categories = ['Normal', 'DOS', 'U2R_AND_R2L']

# Load dataset
# Train
trainsets_1 = {category: pd.read_csv(trainsets_path + category + '_1.csv') for category in train_categories}
trainsets_2 = {category: pd.read_csv(trainsets_path + category + '_2.csv') for category in train_categories}
# Test
testsets = {category: pd.read_csv(testsets_path + category + '.csv') for category in test_categories}

#**2. Tạo bộ dữ liệu cuối từ các bộ dữ liệu đã được chia**
##**Train**
> - **IDS** = Normal_1 + Malicious_1 (label = Attack Category)
> - **GAN**:
 - **GAN-D**: Normal_2
 - **GAN-G**: Malicious_2 (label = Attack Category)

##**Test**
> Đổi label của KDDTest+ thành Attack Category


##**2.1. Train**

In [22]:
# Encode label to Attack Category
for category in train_categories:
    if category == 'U2R' or category == 'R2L':
        trainsets_1[category]['class'] = 'U2R_AND_R2L'
        trainsets_2[category]['class'] = 'U2R_AND_R2L'
    else:
        trainsets_1[category]['class'] = category
        trainsets_2[category]['class'] = category

In [23]:
# Make Trainsets
# Make IDS Trainset
IDS_Trainset = shuffle(pd.concat(trainsets_1.values(), ignore_index=True)).reset_index(drop=True)

# Make GAN Trainset
# Discriminator Trainset
Discriminator_Trainset = trainsets_2['Normal']
# Generator Trainset
Generator_Trainset = shuffle(pd.concat([trainsets_2['DOS'], trainsets_2['U2R'], trainsets_2['R2L']], ignore_index=True)).reset_index(drop=True)


# Statistic & Save
final_trainsets = {'IDS': IDS_Trainset, 'GAN-D': Discriminator_Trainset, 'GAN-G': Generator_Trainset}
for dataset_name, dataset in final_trainsets.items():
    print(f"{10*'*'}{dataset_name}_TRAINSET{10*'*'}")
    print(f" Shape: {dataset.shape}")
    for cat in test_categories:
        print(f"  - Num of {cat}:{len(dataset[dataset['class'] == cat])}")
    dataset_save_path = final_dataset_path+'Trainset/'+dataset_name+'.csv'
    dataset.to_csv(dataset_save_path, index=False)
    print(f"Saved Sucessfully at: {dataset_save_path}")

**********IDS_TRAINSET**********
 Shape: (57157, 42)
  - Num of Normal:33671
  - Num of DOS:22963
  - Num of U2R_AND_R2L:523
Saved Sucessfully at: E:/An toàn thông tin/Code/Dataset/NSL-KDD Processed/Final - For Using/Trainset/IDS.csv
**********GAN-D_TRAINSET**********
 Shape: (33672, 42)
  - Num of Normal:33672
  - Num of DOS:0
  - Num of U2R_AND_R2L:0
Saved Sucessfully at: E:/An toàn thông tin/Code/Dataset/NSL-KDD Processed/Final - For Using/Trainset/GAN-D.csv
**********GAN-G_TRAINSET**********
 Shape: (23488, 42)
  - Num of Normal:0
  - Num of DOS:22964
  - Num of U2R_AND_R2L:524
Saved Sucessfully at: E:/An toàn thông tin/Code/Dataset/NSL-KDD Processed/Final - For Using/Trainset/GAN-G.csv


##**2.2. Test**

In [24]:
# Encode label to Attack Category
for category in test_categories:
    testsets[category]['class'] = category


# Make testset
final_testset = shuffle(pd.concat(testsets.values(), ignore_index=True)).reset_index(drop=True)

# Statistic & Save
print(f"{10*'*'}TESTSET{10*'*'}")
print(f" Shape: {final_testset.shape}")
for cat in test_categories:
    print(f"  - Num of {cat}:{len(final_testset[final_testset['class'] == cat])}")
dataset_save_path = final_dataset_path+'Testset/'+'KDDTest+.csv'
final_testset.to_csv(dataset_save_path, index=False)
print(f"Saved Sucessfully at: {dataset_save_path}")

**********TESTSET**********
 Shape: (20123, 42)
  - Num of Normal:9711
  - Num of DOS:7460
  - Num of U2R_AND_R2L:2952
Saved Sucessfully at: E:/An toàn thông tin/Code/Dataset/NSL-KDD Processed/Final - For Using/Testset/KDDTest+.csv
